<a href="https://colab.research.google.com/github/Dohy-Lee/Do_it-BERT_GPT/blob/main/ner_deploy_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ratsnlp
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 2.8 MB 25.4 MB/s 
     |████████████████████████████████| 57 kB 6.9 MB/s 
     |████████████████████████████████| 582 kB 67.7 MB/s 
     |████████████████████████████████| 596 kB 72.9 MB/s 
     |████████████████████████████████| 419 kB 73.0 MB/s 
     |████████████████████████████████| 141 kB 49.8 MB/s 
     |████████████████████████████████| 880 kB 19.6 MB/s 
     |████████████████████████████████| 101 kB 4.0 MB/s 
     |████████████████████████████████| 3.3 MB 63.6 MB/s 
     |████████████████████████████████| 96 kB 7.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=f8e7d4e813826458e57dc991053a053816145acfc3ca1a5729efd8aacf1fb4f7
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Suc

##인퍼런스 설정

In [5]:
from ratsnlp.nlpbook.ner import NERDeployArguments
args=NERDeployArguments(
    pretrained_model_name='beomi/kcbert-base',
    downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-ner',
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/epoch=1-val_loss=0.20.ckpt
downstream_model_labelmap_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/label_map.txt


##토크나이저 로 드 및 체크포인트 로드

In [8]:
import torch
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)
fine_tuned_model_ckpt=torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device('cpu')
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

##BERT 설정 로드

In [9]:
from transformers import BertConfig, BertForTokenClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)
model=BertForTokenClassification(pretrained_model_config)

##체크포인트 주입

In [10]:
model.load_state_dict({k.replace('model.',""): v for k,v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

##평가 모드로 전환

In [11]:
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

##레이블 맵 작성

In [13]:
labels = [label.strip() for label in open(args.downstream_model_labelmap_fpath, "r").readlines()]
id_to_label = {}
for idx, label in enumerate(labels):
  if "PER" in label:
    label = "인명"
  elif "LOC" in label:
    label = "지명"
  elif "ORG" in label:
    label = "기관명"
  elif "DAT" in label:
    label = "날짜"
  elif "TIM" in label:
    label = "시간"
  elif "DUR" in label:
    label = "기간"
  elif "MNY" in label:
    label = "통화"
  elif "PNT" in label:
    label = "비율"
  elif "NOH" in label:
    label = "기타 수량표현"
  elif "POH" in label:
    label = "기타"
  else:
    label = label
  id_to_label[idx] = label

##인퍼런스

In [15]:
def inference_fn(sentence):
  inputs=tokenizer( #문장을 토큰화하고 인덱싱하되, max_seq_length보다 짧으면 패딩
      [sentence],
      max_length=args.max_seq_length,
      padding='max_length',
      truncation=True,
  )
  with torch.no_grad():
    outputs=model(**{k: torch.tensor(v) for k,v in inputs.items()}) #모델 계산
    probs = outputs.logits[0].softmax(dim=1) #logits에 소프트맥스를 취해 각 토큰이 어떤 개체며에 속하는지 확률 구하기
    top_probs, preds = torch.topk(probs, dim=1,k=1) #각 토큰이 속하는 개체명 확률 분포 가운데 가장 높은 확률값과 그에 속하는 개체명 인덱스 구하기
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]) #토큰 인덱스 시퀀스 → 토큰 시퀀스로 변환
    predicted_tags = [id_to_label[pred.item()] for pred in preds] #개체명 인덱스 시퀀스 → 개체명 시퀀스로 변환
    result = []
    for token, predicted_tag, top_prob in zip(tokens, predicted_tags, top_probs): #[CLS],[SEP],[PAD]를 제외한 토큰 각각에 대해 모델이 예측한 개체명(predicted_tag)과 그 확률값(top_prob) 반환
      if token not in [tokenizer.pad_token, tokenizer.cls_token, tokenizer.sep_token]:
        token_result={
            'token':token,
            'predicted_tag':predicted_tag,
            'top_prob':str(round(top_prob[0].item(),4))
        }
        result.append(token_result)
  return{
      'sentence':sentence,
      'result':result,
  }

##웹 서비스


In [17]:
!mkdir /root/.ngrok2 && echo "authtoken: 개인토큰키" > /root/.ngrok2/ngrok.yml


mkdir: cannot create directory ‘/root/.ngrok2’: File exists


In [ ]:
from ratsnlp.nlpbook.ner import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.ner.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://78ef-34-124-246-116.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 20:42:22] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 20:42:23] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 20:42:33] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 20:42:44] "POST /api HTTP/1.1" 200 -
